In [16]:
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore


In [49]:
#load data
atlas_path = "/home/gabridele/Desktop/irbio_folder/spreading_dynamics_clinical/derivatives/templates/Schaefer2018_400Parcels_Tian_Subcortex_S4_1mm_2009c_NLinAsymm.nii.gz"
atlas_img = nib.load(atlas_path)
atlas_data = atlas_img.get_fdata()

def show_slices(slices):
   """ Function to display row of image slices """
   fig, axes = plt.subplots(1, len(slices))
   for i, slice in enumerate(slices):
       axes[i].imshow(slice.T, cmap="gray", origin="lower") 

In [18]:
# Load data
vols = pd.read_csv("irbio_folder/spreading_dynamics_clinical/concat_vols.tsv", sep='\t')
vols.index.name = 'Parcels'
vols.index += 1

vols = vols.reindex(sorted(vols.columns), axis=1)

In [36]:
zscored_vols = vols.apply(zscore, axis=1)

roi_iqr = zscored_vols.apply(lambda x: x.quantile(0.75) - x.quantile(0.25), axis=1)
print(roi_iqr)

Parcels
1      1.084825
2      1.184077
3      1.452108
4      1.568265
5      1.185797
         ...   
450    1.342307
451    1.317956
452    1.475362
453    1.052464
454    1.347649
Length: 454, dtype: float64


In [50]:
stats_data = np.zeros_like(atlas_data)

for roi_index in range(1, len(roi_iqr_vector) + 1):
    roi_indices = np.where(atlas_data == roi_index)
    stats_data[roi_indices] = roi_iqr_vector[roi_index - 1]

stats_img = nib.Nifti1Image(stats_data, atlas_img.affine, header=atlas_img.header)
nib.save(stats_img, 'roi_iqr_values_mapped.nii.gz')